In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import itertools

* Sklearn

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score

Pytorch

In [3]:
import torch

from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.nn import Sequential, Linear, SAGEConv, to_hetero
from torch.nn import ReLU
import torch.nn.functional as F

* Paths and warnings

In [4]:
import warnings
warnings.filterwarnings("ignore")

path = "../data/eda_generated_data/"
output_path = "../data/graph_data/"

In [11]:
from graphs_management import graph_test
model = torch.load('../data/models/sage_heterogen_model.pt')

* Load data

In [6]:
def load_pickle(path, file_name):
    with open(path + file_name, 'rb') as f: return pickle.load(f)

path = "../data/graph_data/"
eda_path = "../data/eda_generated_data/"
output_path = "../data/models/"

In [7]:
training_graph = torch.load(path + 'training_graph.pt')
df_test = load_pickle(eda_path, "df_test.pkl")

In [8]:
eda_path = "../data/eda_generated_data/"

training_graph = torch.load(path + 'training_graph.pt')
graph_df = load_pickle(path, 'graph_df.pkl')
df_train = load_pickle(eda_path, 'df_train.pkl')
scaler = load_pickle(eda_path, 'scaler.pkl')

In [24]:
training_graph_new = graph_test(df_test.iloc[:10000])

In [25]:
model.eval()
out = model(training_graph_new.x_dict, training_graph_new.edge_index_dict)

mask = training_graph_new['users'].test_mask
pred = out['users'][mask].max(1)[1]

In [23]:
roc_auc_score(df_test.iloc[:1000].TARGET, pred), f1_score(df_test.iloc[:1000].TARGET, pred), precision_score(df_test.iloc[:1000].TARGET, pred), recall_score(df_test.iloc[:1000].TARGET, pred)

(0.6785287081339714, 0.28723404255319146, 0.1875, 0.6136363636363636)

In [26]:
roc_auc_score(df_test.iloc[:10000].TARGET, pred), f1_score(df_test.iloc[:10000].TARGET, pred), precision_score(df_test.iloc[:10000].TARGET, pred), recall_score(df_test.iloc[:10000].TARGET, pred)

(0.6471225888729476,
 0.24756229685807152,
 0.15912256267409472,
 0.5573170731707318)